In [1]:
from collections import OrderedDict
import numpy as np
import cv2
import argparse
import dlib
import imutils
import math

In [2]:
facial_features_cordinates={}

In [3]:
#define a dictionary that maps the indexes of the facial
# landmarks to specific face regions
FACIAL_LANDMARKS_INDEXES = OrderedDict([
    ("Mouth", (48, 68)),
    ("Right_Eyebrow", (17, 22)),
    ("Left_Eyebrow", (22, 27)),
    ("Right_Eye", (36, 42)),
    ("Left_Eye", (42, 48)),
    ("Nose", (27, 36)),
    ("Jaw", (0, 17))
])

In [4]:
def shape_to_numpy_array(shape, dtype="int"):
    

    # initialize the list of (x, y)-coordinates
    coordinates = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coordinates[i] = (shape.part(i).x, shape.part(i).y)

    

    # return the list of (x, y)-coordinates
    return coordinates

In [5]:
def visualize_facial_landmarks(image, shape, colors=None, alpha=0.75):

    

    # create two copies of the input image -- one for the
    # overlay and one for the final output image
    overlay = image.copy()
    output = image.copy()

    

    # if the colors list is None, initialize it with a unique

    # color for each facial landmark region
    if colors is None:
        colors = [(19, 199, 109), (79, 76, 240), (230, 159, 23),
                  (168, 100, 168), (158, 163, 32),
                  (163, 38, 32), (180, 42, 220)]

    

    # loop over the facial landmark regions individually
    for (i, name) in enumerate(FACIAL_LANDMARKS_INDEXES.keys()):
        

        # grab the (x, y)-coordinates associated with the
        # face landmark
        (j, k) = FACIAL_LANDMARKS_INDEXES[name]

        

        pts = shape[j:k]
        facial_features_cordinates[name] = pts

        

    # apply the transparent overlay
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)

    # return the output image
    #print(facial_features_cordinates)
    return output

In [6]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [7]:
def __angle_between(p1,p2): #두점 사이의 각도:(getAngle3P계산용) 시계방향으로 계산.P1-(0,0)
    ang1=np.arctan2(*p1[::-1])
    ang2=np.arctan2(*p2[::-1])
    res=np.rad2deg((ang1-ang2)%(2*np.pi))
    return res

In [8]:
def getAngle4P(p1, p2, p3, p4, direction="CW"): # 네점 사이의 각도 4-3 / 2-1: 1->2-> .... 4->3 으로 그려지는 각도
        pt1 = (p1[0] - p2[0], p1[1] - p2[1])              # 3 <- 4 <- V <-2<-1 으로 그려지는 각도
        pt2 = (p3[0] - p4[0], p3[1] - p4[1])        
        res = __angle_between(pt1, pt2)
        res = (res + 360) % 360
        if direction == "CCW":    #반시계방향
            res = (360 - res) % 360
        return res

In [9]:
def guansang(address):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    #이미지 입력(주소 받아서)
    image = cv2.imread(address)

    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    rects = detector(gray, 1)
    #얼굴에 특징 점 지정작업
    for (i, rect) in enumerate(rects):
    

    # determine the facial landmarks for the face region, then
    # convert the landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = shape_to_numpy_array(shape)

    

        output = visualize_facial_landmarks(image, shape)
    
    #턱밑각 추출 
    jaw=facial_features_cordinates["Jaw"]
    
    point1=jaw[5]
    point2=jaw[6]
    point3=jaw[10]
    point4=jaw[11]
    
    face_angle=getAngle4P((point1[1],point1[0]),(point2[1],point2[0]),(point4[1],point4[0]),(point3[1],point3[0]),"CW")
    
    jaw_result=(180-face_angle)/2 
    
    #눈,코 비율
    right_Eye=facial_features_cordinates["Right_Eye"]
    left_Eye=facial_features_cordinates["Left_Eye"]
    nose=facial_features_cordinates["Nose"]
    
    left_h_rate=(jaw[8][1]-left_Eye[5][1])/(jaw[8][1]-left_Eye[2][1])
    left_w_rate=(jaw[8][0]-left_Eye[0][0])/(jaw[8][0]-left_Eye[3][0])
    right_h_rate=(jaw[8][1]-right_Eye[5][1])/(jaw[8][1]-right_Eye[2][1])
    right_w_rate=(right_Eye[3][0]-jaw[8][0])/(right_Eye[0][0]-jaw[8][0])
    nose_h_rate=(jaw[8][1]-nose[6][1])/(jaw[8][1]-nose[0][1])
    nose_w_rate=(nose[4][0]-jaw[0][0])/(nose[8][0]-jaw[0][0])
    
    result=[jaw_result,left_h_rate,left_w_rate,right_h_rate,right_w_rate,nose_h_rate,nose_w_rate]
    
    return result

In [10]:
j_angle=[]
l_h_rate=[]
l_w_rate=[]
r_h_rate=[]
r_w_rate=[]
n_h_rate=[]
n_w_rate=[]

In [11]:
guansang("../data/ones2.jpg")

[45.774078849489,
 0.9428571428571428,
 0.4067796610169492,
 0.9367816091954023,
 0.4782608695652174,
 0.5614035087719298,
 0.6929133858267716]

In [12]:
fname=["./prject_data/face/{}_m.jpg".format(i) for i in range(1,400)]

for fname in fname:
    result=guansang(fname)
    j_angle.append(result[0])
    l_h_rate.append(result[1])
    l_w_rate.append(result[2])
    r_h_rate.append(result[3])
    r_w_rate.append(result[4])
    n_h_rate.append(result[5])
    n_w_rate.append(result[6])

In [13]:
j_angle

[41.18592516570965,
 42.14470343125018,
 25.428006792714484,
 34.24928283797607,
 37.65839012380201,
 37.43799634584471,
 35.27998258591188,
 43.63684450304689,
 43.63684450304689,
 29.518121733963255,
 39.00729294046016,
 35.44845455437462,
 30.540825978406843,
 34.99202019855869,
 37.2503623117496,
 36.489737194240064,
 31.363814727188895,
 32.47119229084848,
 36.270398682186425,
 45.0,
 33.14190527431643,
 43.49360624790833,
 30.1275593515289,
 45.0,
 25.67009587295496,
 39.345033762989885,
 35.56550027246931,
 27.51578886618063,
 39.28940686250036,
 35.78252558853899,
 32.82945503165374,
 42.14470343125018,
 31.973252344754542,
 29.278596464767816,
 29.433448414747232,
 38.65980825409011,
 31.973252344754542,
 38.079652254172174,
 42.27368900609375,
 36.86989764584402,
 30.1275593515289,
 34.09929525682409,
 34.99202019855869,
 29.278596464767816,
 42.14470343125018,
 36.86989764584402,
 20.345667541737555,
 33.08062990841415,
 47.59721445386742,
 39.42855507083033,
 34.85872045554

In [14]:
len(j_angle)

399

In [15]:
face_data={"jaw_angle":j_angle,"left_height":l_h_rate,"left_width":l_w_rate,
        "right_height":r_h_rate,"right_width":r_w_rate,"nose_height":n_h_rate,
         "nose_width":n_w_rate}

In [16]:
import pandas as pd
face_df=pd.DataFrame(face_data)

In [17]:
face_df

,jaw_angle,left_height,left_width,right_height,right_width,nose_height,nose_width
0,41.185925,0.945946,0.392857,0.932432,0.466667,0.575342,0.730769
1,42.144703,0.959596,0.454545,0.959596,0.485714,0.560000,0.671642
2,25.428007,0.929577,0.423077,0.929577,0.464286,0.605634,0.714286
3,34.249283,0.931034,0.419355,0.941176,0.470588,0.541176,0.718750
4,37.658390,0.902174,0.473684,0.913978,0.322581,0.534091,0.661765
...,...,...,...,...,...,...,...
394,40.638310,0.959596,0.500000,0.950495,0.437500,0.564356,0.641791
395,30.676192,0.931373,0.523810,0.941176,0.363636,0.574257,0.671233
396,39.007293,0.947368,0.513514,0.947917,0.437500,0.572917,0.692308
397,34.238700,0.941748,0.457143,0.941176,0.428571,0.607843,0.688312


In [18]:
face_df.to_csv("face_data.csv")

In [19]:
face_df["left_eye"]=face_df["left_height"]*face_df["left_width"]
face_df["right_eye"]=face_df["right_height"]*face_df["right_width"]
face_df["nose"]=face_df["nose_height"]*face_df["nose_width"]

In [20]:
print('왼쪽눈 비율의 최대 값: {}'.format(np.max(face_df["left_eye"])))
print('왼쪽눈 비율의 평균 값: {:.2f}'.format(np.mean(face_df["left_eye"])))
print('왼쪽눈 비율의 표준편차: {:.2f}'.format(np.std(face_df["left_eye"])))
print('왼쪽눈 비율의 중간 값: {}'.format(np.median(face_df["left_eye"])))
print('왼쪽눈 비율의 제 1 사분위: {}'.format(np.percentile(face_df["left_eye"], 25)))
print('왼쪽눈 비율의 제 3 사분위: {}'.format(np.percentile(face_df["left_eye"], 75)))

왼쪽눈 비율의 최대 값: 0.5749271137026238
왼쪽눈 비율의 평균 값: 0.41
왼쪽눈 비율의 표준편차: 0.05
왼쪽눈 비율의 중간 값: 0.41143423661599465
왼쪽눈 비율의 제 1 사분위: 0.3804347826086956
왼쪽눈 비율의 제 3 사분위: 0.4394907407407408


In [21]:
print('오른쪽눈 비율의 최대 값: {}'.format(np.max(face_df["right_eye"])))
print('오른쪽눈 비율의 평균 값: {:.2f}'.format(np.mean(face_df["right_eye"])))
print('오른쪽눈 비율의 표준편차: {:.2f}'.format(np.std(face_df["right_eye"])))
print('오른쪽눈 비율의 중간 값: {}'.format(np.median(face_df["right_eye"])))
print('오른쪽눈 비율의 제 1 사분위: {}'.format(np.percentile(face_df["right_eye"], 25)))
print('오른쪽눈 비율의 제 3 사분위: {}'.format(np.percentile(face_df["right_eye"], 75)))

오른쪽눈 비율의 최대 값: 0.5605640314378179
오른쪽눈 비율의 평균 값: 0.42
오른쪽눈 비율의 표준편차: 0.05
오른쪽눈 비율의 중간 값: 0.4238095238095238
오른쪽눈 비율의 제 1 사분위: 0.3975054020682204
오른쪽눈 비율의 제 3 사분위: 0.45063550673930786


In [22]:
print('코 비율의 최대 값: {}'.format(np.max(face_df["nose"])))
print('코 비율의 평균 값: {:.2f}'.format(np.mean(face_df["nose"])))
print('코 비율의 표준편차: {:.2f}'.format(np.std(face_df["nose"])))
print('코 비율의 중간 값: {}'.format(np.median(face_df["nose"])))
print('코 비율의 제 1 사분위: {}'.format(np.percentile(face_df["nose"], 25)))
print('코 비율의 제 3 사분위: {}'.format(np.percentile(face_df["nose"], 75)))

코 비율의 최대 값: 0.47016645326504486
코 비율의 평균 값: 0.40
코 비율의 표준편차: 0.02
코 비율의 중간 값: 0.40012702445220705
코 비율의 제 1 사분위: 0.38380308536864305
코 비율의 제 3 사분위: 0.4150151153383838


one - 계란형, 둥근형
tri - 역삼각형
rec - 사각형,마름모형,긴형

In [23]:
min(face_df["jaw_angle"])

19.49549702125273

In [24]:
face_df[face_df["jaw_angle"]==19.49549702125273]

,jaw_angle,left_height,left_width,right_height,right_width,nose_height,nose_width,left_eye,right_eye,nose
366,19.495497,0.943925,0.463415,0.93578,0.388889,0.59633,0.666667,0.437429,0.363914,0.397554


In [25]:
max(face_df["jaw_angle"])

51.13238686394621

In [26]:
face_df[face_df["jaw_angle"]==51.13238686394621]

,jaw_angle,left_height,left_width,right_height,right_width,nose_height,nose_width,left_eye,right_eye,nose
398,51.132387,0.941176,0.547619,0.942857,0.366667,0.544554,0.671642,0.515406,0.345714,0.365746


In [27]:
face_df["jaw_angle"][71]

34.24928283797607

In [28]:
face_df["jaw_angle"][76]

31.363814727188895

In [29]:
face_df["jaw_angle"][48]

47.59721445386742

In [30]:
face_df["jaw_angle"][49]

39.42855507083033

In [31]:
face_df["jaw_angle"][50]

34.858720455541686

In [32]:
face_df["jaw_angle"][55] #각진느낌

26.53794816741916

In [33]:
face_df["jaw_angle"][56] #각진느낌

35.27998258591188

In [34]:
face_df["jaw_angle"][58] #계란형 59번사진

43.49360624790833

In [35]:
face_df["jaw_angle"][60] #61번사진 각진느낌

27.51578886618063

In [36]:
face_df["jaw_angle"][53]#각진느낌 54번사진

31.607502246248913

In [37]:
face_df["jaw_angle"][62]

36.174937890034926

In [38]:
face_df["jaw_angle"][78] #역삼각형 느낌

39.28940686250036

In [39]:
face_df["jaw_angle"][86] #각진 느낌 87번사진

29.95412900612763

In [40]:
face_df["jaw_angle"][104]

34.77292201301481

In [41]:
face_df["jaw_angle"][109]

45.0

In [42]:
face_df["jaw_angle"][120] #각진

20.345667541737555

In [43]:
face_df["jaw_angle"][133] #각진

27.432903471542204

In [44]:
face_df["jaw_angle"][151]

37.83864001089822

In [45]:
face_df["jaw_angle"][233]

40.85142751508158

In [46]:
face_df["jaw_angle"][248]

39.00729294046016

In [47]:
face_df["jaw_angle"][269]

42.14470343125018

In [48]:
face_df["jaw_angle"][339]

45.0

In [49]:
guansang('../data/triangle.jfif')

[40.25254007937491,
 0.9365079365079365,
 0.4583333333333333,
 0.9354838709677419,
 0.4117647058823529,
 0.5683060109289617,
 0.7248322147651006]

- 43~46 계란형
- 46 초과 긴형
- 38~43 역삼각형
- 38 미만 각진얼굴

In [50]:
face_df["jaw_angle"][253] #긴형

46.5063937520917

In [51]:
face_df["jaw_angle"][275]

35.94498825004973

In [193]:
def do_guansang(address):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    #이미지 입력(주소 받아서)
    image = cv2.imread(address)

    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    rects = detector(gray, 1)
    #얼굴에 특징 점 지정작업
    for (i, rect) in enumerate(rects):
    

    # determine the facial landmarks for the face region, then
    # convert the landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = shape_to_numpy_array(shape)

    

        output = visualize_facial_landmarks(image, shape)
    
    #턱밑각 추출 
    jaw=facial_features_cordinates["Jaw"]
    
    point1=jaw[5]
    point2=jaw[6]
    point3=jaw[10]
    point4=jaw[11]
    
    face_angle=getAngle4P((point1[1],point1[0]),(point2[1],point2[0]),(point4[1],point4[0]),(point3[1],point3[0]),"CW")
    
    jaw_angle=(180-face_angle)/2 
    
    if jaw_angle<38:
        jaw_result="R"
    elif jaw_angle<43:
        jaw_result="T"
    elif jaw_angle<46:
        jaw_result="E"
    else:
        jaw_result="R"
    
    #눈,코 비율
    right_Eye=facial_features_cordinates["Right_Eye"]
    left_Eye=facial_features_cordinates["Left_Eye"]
    nose=facial_features_cordinates["Nose"]
    
    left_h_rate=(jaw[8][1]-left_Eye[5][1])/(jaw[8][1]-left_Eye[2][1])
    left_w_rate=(jaw[8][0]-left_Eye[0][0])/(jaw[8][0]-left_Eye[3][0])
    right_h_rate=(jaw[8][1]-right_Eye[5][1])/(jaw[8][1]-right_Eye[2][1])
    right_w_rate=(right_Eye[3][0]-jaw[8][0])/(right_Eye[0][0]-jaw[8][0])
    nose_h_rate=(jaw[8][1]-nose[6][1])/(jaw[8][1]-nose[0][1])
    nose_w_rate=(nose[4][0]-jaw[0][0])/(nose[8][0]-jaw[0][0])
    
#     print(jaw_angle)
#     print(left_h_rate*left_w_rate)
#     print(right_h_rate*right_w_rate)
#     print(nose_h_rate*nose_w_rate)
    
    
    
    if left_h_rate*left_w_rate<0.3804347826086956:
        left_rate="LA"
    elif left_h_rate*left_w_rate<0.41143423661599465:
        left_rate="LB"
    elif left_h_rate*left_w_rate<0.4394907407407408:
        left_rate="LC"
    else :
        left_rate="LD"
        
    if right_h_rate*right_w_rate<0.3975054020682204:
        right_rate="RA"
    elif right_h_rate*right_w_rate<0.4238095238095238:
        right_rate="RB"
    elif right_h_rate*right_w_rate<0.45063550673930786:
        right_rate="RC"
    else :
        right_rate="RD"
    
    if nose_h_rate*nose_w_rate<0.38380308536864305:
        nose_rate="NA"
    elif nose_h_rate*nose_w_rate<0.40012702445220705:
        nose_rate="NB"
    elif nose_h_rate*nose_w_rate<0.4150151153383838:
        nose_rate="NC"
    else:
        nose_rate="ND"
    
    if left_Eye[3][1]>left_Eye[0][1]:
        eye_position="U"
    else:
        eye_position="D"
    
    result=[jaw_result,left_rate,right_rate,nose_rate,eye_position]
    
    
    return result

- 남T, 여O 매우좋음
- 남T, 여R 조금 좋음
- 남R, 여O 조금 좋음
- 남R, 여T 조금 좋음
- 남R, 여R 조금 좋음
- 남O, 여R 조금 좋음
- 남O, 여T 조금 좋음

In [105]:
if result[0][0]=="T" and result[1][0]=="O":
    count1=100
elif result[0][0]=="T" and result[1][0]=="R":
    count1=50
elif result[0][0]=="R" and result[1][0]=="O":
    count1=50
elif result[0][0]=="R" and result[1][0]=="T":
    count1=50
elif result[0][0]=="R" and result[1][0]=="R":
    count1=50
elif result[0][0]=="O" and result[1][0]=="R":
    count1=50
elif result[0][0]=="O" and result[1][0]=="T":
    count1=50
else:
    count1=30

In [106]:
if result[0][1]=="LA" and result[1][1]=="LA":
    count2=30
elif result[0][1]=="LA" and result[1][1]=="LB":
    count2=30
elif result[0][1]=="LA" and result[1][1]=="LC":
    count2=30
elif result[0][1]=="LA" and result[1][1]=="LD":
    count2=100
elif result[0][1]=="LB" and result[1][1]=="LA":
    count2=30
elif result[0][1]=="LB" and result[1][1]=="LB":
    count2=30
elif result[0][1]=="LB" and result[1][1]=="LC":
    count2=30
elif result[0][1]=="LB" and result[1][1]=="LD":
    count2=30
elif result[0][1]=="LC" and result[1][1]=="LA":
    count2=30
elif result[0][1]=="LC" and result[1][1]=="LB":
    count2=30
elif result[0][1]=="LC" and result[1][1]=="LC":
    count2=30
elif result[0][1]=="LC" and result[1][1]=="LD":
    count2=30
elif result[0][1]=="LD" and result[1][1]=="LA":
    count2=100
elif result[0][1]=="LD" and result[1][1]=="LB":
    count2=30
elif result[0][1]=="LD" and result[1][1]=="LC":
    count2=30
elif result[0][1]=="LD" and result[1][1]=="LD":
    count2=30

In [103]:
if result[0][2]=="RA" and result[1][2]=="RA":
    count3=30
elif result[0][2]=="RA" and result[1][2]=="RB":
    count3=30
elif result[0][2]=="RA" and result[1][2]=="RC":
    count3=30
elif result[0][2]=="RA" and result[1][2]=="RD":
    count3=100
elif result[0][2]=="RB" and result[1][2]=="RA":
    count3=30
elif result[0][2]=="RB" and result[1][2]=="RB":
    count3=30
elif result[0][2]=="RB" and result[1][2]=="RC":
    count3=30
elif result[0][2]=="RB" and result[1][2]=="RD":
    count3=30
elif result[0][2]=="RC" and result[1][2]=="RA":
    count3=30
elif result[0][2]=="RC" and result[1][2]=="RB":
    count3=30
elif result[0][2]=="RC" and result[1][2]=="RC":
    count3=30
elif result[0][2]=="RC" and result[1][2]=="RD":
    count3=30
elif result[0][2]=="RD" and result[1][2]=="RA":
    count3=100
elif result[0][2]=="RD" and result[1][2]=="RB":
    count3=30
elif result[0][2]=="RD" and result[1][2]=="RC":
    count3=30
elif result[0][2]=="RD" and result[1][2]=="RD":
    count3=30

In [104]:
if result[0][3]=="NA" and result[1][3]=="NA":
    count4=30
elif result[0][3]=="NA" and result[1][3]=="NB":
    count4=30
elif result[0][3]=="NA" and result[1][3]=="NC":
    count4=30
elif result[0][3]=="NA" and result[1][3]=="ND":
    count4=100
elif result[0][3]=="NB" and result[1][3]=="NA":
    count4=30
elif result[0][3]=="NB" and result[1][3]=="NB":
    count4=30
elif result[0][3]=="NB" and result[1][3]=="NC":
    count4=30
elif result[0][3]=="NB" and result[1][3]=="ND":
    count4=30
elif result[0][3]=="NC" and result[1][3]=="NA":
    count4=30
elif result[0][3]=="NC" and result[1][3]=="NB":
    count4=30
elif result[0][3]=="NC" and result[1][3]=="NC":
    count4=30
elif result[0][3]=="NC" and result[1][3]=="ND":
    count4=30
elif result[0][3]=="ND" and result[1][3]=="NA":
    count4=100
elif result[0][3]=="ND" and result[1][3]=="NB":
    count4=30
elif result[0][3]=="ND" and result[1][3]=="NC":
    count4=30
elif result[0][3]=="ND" and result[1][3]=="ND":
    count4=30

In [161]:
result=[]

In [157]:
result.append(do_guansang("wun.jpg"))
result.append(do_guansang("sana.jpg"))
for i in range(len(result)):
    print(result[i])

['T', 'LD', 'RA', 'NC']
['T', 'LC', 'RA', 'NA']


In [143]:
result[0][0]

'T'

In [144]:
result[0][1]

'LD'

In [145]:
result[0][2]

'RA'

In [159]:
result.append(do_guansang("jisung2.jpg"))
result.append(do_guansang("seo.jpg"))
for i in range(len(result)):
    print(result[i])

['T', 'LC', 'RA', 'NA']
['R', 'LA', 'RB', 'NA']


In [162]:
result.append(do_guansang("s1.png"))
result.append(do_guansang("s2.png"))
for i in range(len(result)):
    print(result[i])

['E', 'LD', 'RA', 'NA']
['T', 'LD', 'RA', 'NA']


In [192]:
result_number=count1*0.25+count2*0.125+count3*0.125+count4*0.25+count5*0.25

NameError: name 'count5' is not defined

In [163]:
count1

50

In [138]:
count2

30

In [139]:
count3

30

In [140]:
count4

30

In [119]:
result_number

40.0

In [78]:
result[0][0]

'E'

In [79]:
result[0][1]

'LB'

In [80]:
result[0][2]

'RC'

In [81]:
result

[['E', 'LB', 'RC', 'NB'], ['R', 'LA', 'RB', 'NA']]

In [165]:
matching("jisung2.jpg","seo.jpg")

[['T', 'LD', 'RA', 'NA'], ['R', 'LA', 'RB', 'NA']]

In [164]:
def matching(adress1,adress2):
    
    result=[]
    result.append(do_guansang(adress1))
    result.append(do_guansang(adress2))
    
    return result

In [228]:
def g_count(man_adress1,female_adress2):
    result=matching(man_adress1,female_adress2)
    
    #턱
    if result[0][0]=="T" and result[1][0]=="O":
        count1=100
    elif result[0][0]=="T" and result[1][0]=="R":
        count1=50
    elif result[0][0]=="R" and result[1][0]=="O":
        count1=50
    elif result[0][0]=="R" and result[1][0]=="T":
        count1=50
    elif result[0][0]=="R" and result[1][0]=="R":
        count1=50
    elif result[0][0]=="O" and result[1][0]=="R":
        count1=50
    elif result[0][0]=="O" and result[1][0]=="T":
        count1=50
    else:
        count1=30
    
    #왼눈 10,40,70,100
    if result[0][1]=="LA" and result[1][1]=="LA":
        count2=10
    elif result[0][1]=="LA" and result[1][1]=="LB":
        count2=40
    elif result[0][1]=="LA" and result[1][1]=="LC":
        count2=70
    elif result[0][1]=="LA" and result[1][1]=="LD":
        count2=100
    elif result[0][1]=="LB" and result[1][1]=="LA":
        count2=40
    elif result[0][1]=="LB" and result[1][1]=="LB":
        count2=10
    elif result[0][1]=="LB" and result[1][1]=="LC":
        count2=40
    elif result[0][1]=="LB" and result[1][1]=="LD":
        count2=70
    elif result[0][1]=="LC" and result[1][1]=="LA":
        count2=70
    elif result[0][1]=="LC" and result[1][1]=="LB":
        count2=40
    elif result[0][1]=="LC" and result[1][1]=="LC":
        count2=10
    elif result[0][1]=="LC" and result[1][1]=="LD":
        count2=40
    elif result[0][1]=="LD" and result[1][1]=="LA":
        count2=100
    elif result[0][1]=="LD" and result[1][1]=="LB":
        count2=70
    elif result[0][1]=="LD" and result[1][1]=="LC":
        count2=40
    elif result[0][1]=="LD" and result[1][1]=="LD":
        count2=10
    
    #오눈
    if result[0][2]=="RA" and result[1][2]=="RA":
        count3=10
    elif result[0][2]=="RA" and result[1][2]=="RB":
        count3=40
    elif result[0][2]=="RA" and result[1][2]=="RC":
        count3=70
    elif result[0][2]=="RA" and result[1][2]=="RD":
        count3=100
    elif result[0][2]=="RB" and result[1][2]=="RA":
        count3=40
    elif result[0][2]=="RB" and result[1][2]=="RB":
        count3=10
    elif result[0][2]=="RB" and result[1][2]=="RC":
        count3=40
    elif result[0][2]=="RB" and result[1][2]=="RD":
        count3=70
    elif result[0][2]=="RC" and result[1][2]=="RA":
        count3=70
    elif result[0][2]=="RC" and result[1][2]=="RB":
        count3=40
    elif result[0][2]=="RC" and result[1][2]=="RC":
        count3=10
    elif result[0][2]=="RC" and result[1][2]=="RD":
        count3=40
    elif result[0][2]=="RD" and result[1][2]=="RA":
        count3=100
    elif result[0][2]=="RD" and result[1][2]=="RB":
        count3=70
    elif result[0][2]=="RD" and result[1][2]=="RC":
        count3=40
    elif result[0][2]=="RD" and result[1][2]=="RD":
        count3=10
        
    #코
    if result[0][3]=="NA" and result[1][3]=="NA":
        count4=10
    elif result[0][3]=="NA" and result[1][3]=="NB":
        count4=40
    elif result[0][3]=="NA" and result[1][3]=="NC":
        count4=70
    elif result[0][3]=="NA" and result[1][3]=="ND":
        count4=100
    elif result[0][3]=="NB" and result[1][3]=="NA":
        count4=40
    elif result[0][3]=="NB" and result[1][3]=="NB":
        count4=10
    elif result[0][3]=="NB" and result[1][3]=="NC":
        count4=40
    elif result[0][3]=="NB" and result[1][3]=="ND":
        count4=70
    elif result[0][3]=="NC" and result[1][3]=="NA":
        count4=70
    elif result[0][3]=="NC" and result[1][3]=="NB":
        count4=40
    elif result[0][3]=="NC" and result[1][3]=="NC":
        count4=10
    elif result[0][3]=="NC" and result[1][3]=="ND":
        count4=40
    elif result[0][3]=="ND" and result[1][3]=="NA":
        count4=100
    elif result[0][3]=="ND" and result[1][3]=="NB":
        count4=70
    elif result[0][3]=="ND" and result[1][3]=="NC":
        count4=40
    elif result[0][3]=="ND" and result[1][3]=="ND":
        count4=10
    
    #눈꼬리
    if result[0][4]=="U" and result[1][4]=="D":
        count5=100
    elif result[0][4]=="D" and result[1][4]=="U":
        count5=100
    else:
        count5=10
    
    #점수합산
    before_result=count1*0.4+count2*0.125+count3*0.125+count4*0.25+count5*0.1
    result_number=math.log(before_result)*21.71472409516259
    
    return result_number

In [195]:
g_count("jisung2.jpg","seo.jpg")

57.5

In [196]:
g_count("wun.jpg","sana.jpg")

33.75

In [209]:
g_count("s1.png","s2.png")

18.0

In [210]:
g_count("choi.png","ha.png")

56.0

In [211]:
g_count("choi2.png","ha.png")

51.75

In [207]:
g_count("jisung2.jpg","bo.png")

69.0

In [233]:
math.log(18)

2.8903717578961645

In [234]:
math.log(56)

4.02535169073515

In [201]:
g_count("wun.jpg","nah.png")

52.5

In [237]:
math.log(100)*21.71472409516259

100.00000000000001

In [224]:
a=g_count("s1.png","s2.png")

In [225]:
a

18.0

In [226]:
log=math.log(a)*21.71472409516259

In [227]:
log

62.7636252551653

In [218]:
b=g_count("jisung2.jpg","bo.png")

In [219]:
log2=math.log(b)*21.71472409516259

In [220]:
log2

80.63919283598678

In [221]:
c=g_count("choi.png","ha.png")

In [222]:
log3=math.log(c)*21.71472409516259

In [223]:
log3

87.40940135031003

In [229]:
g_count("s1.png","s2.png")

62.7636252551653

In [230]:
g_count("jisung2.jpg","bo.png")

87.98339223448153

In [231]:
g_count("choi.png","ha.png")

87.40940135031003

In [238]:
g_count("hun.jpg","bo.png")

87.40940135031003

In [240]:
g_count("hun.jpg","suzi.png")

84.06206186877937

In [241]:
g_count("hun.jpg","seo.jpg")

87.40940135031003

In [242]:
g_count("junsuck.jpg","suzi.png")

78.55631385421557

In [243]:
g_count("junsuck.jpg","seo.jpg")

82.53965198259654

In [244]:
g_count("junsuck.jpg","bo.png")

85.90431473915459

In [245]:
g_count("junsuck.jpg","nara.png")

88.81689548100877

In [246]:
g_count("junsuck.jpg","bin.jfif")

78.55631385421557

In [247]:
g_count("junsuck.jpg","minjung.jfif")

78.55631385421557

In [248]:
g_count("junsuck.jpg","gg.PNG")

84.28708693011318

In [249]:
g_count("junsuck.jpg","swift.PNG")

80.63919283598678